In [1]:
# import classes for python application
from pyspark.sql import SparkSession, DataFrame, functions, Row
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *

from pyspark.sql.functions import explode 
from pyspark.sql.functions import split
from pyspark.sql.functions import to_json, struct, col

import pandas as pd
import os
import sys
import pyarrow as pa
import pyarrow.parquet as pq

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'


# create spark session, configuration and context
spark = SparkSession.builder.appName("DataFrame").getOrCreate()
conf = SparkConf().setMaster("local").setAppName("Assignment 7")
sc = SparkContext.getOrCreate(conf=conf)

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/f_dev/.ivy2/cache
The jars for the packages stored in: /home/f_dev/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-87f63c4d-dd37-455b-b0c9-9a6b9742a2af;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-loggi

NetCat Command -> ```nc -lk 9999```

In [2]:
# listen for updates
input_words = (spark
.readStream.format("kafka.bootstrap.servers").option("localhost:9092")
.option("port", 9999).load())

# get word count
words = input_words.select(split(col("value"), "\\s").alias("word"))
counts = words.groupBy("word").count()
checkpointDir = "./checkpoint3" 

# run the query
streamingQuery = (counts
.writeStream.format("console").outputMode("complete")
.trigger(processingTime="5 second")
.option("checkpointLocation", checkpointDir).start())

streamingQuery.awaitTermination()

22/03/27 02:15:35 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
22/03/27 02:15:38 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/f_dev/.local/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/f_dev/.local/lib/python3.9/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [5]:
from confluent_kafka.admin import AdminClient, NewTopic

admin_client = AdminClient({
    "bootstrap.servers": "localhost:9092"
})

topic_list = []
topic_list.append(NewTopic("Topic1", 1, 1))
admin_client.create_topics(topic_list)

%3|1648348053.852|FAIL|rdkafka#producer-3| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT)


{'Topic1': <Future at 0x7f31880fd4f0 state=running>}

%3|1648348054.852|FAIL|rdkafka#producer-3| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT, 1 identical error(s) suppressed)
%3|1648348084.861|FAIL|rdkafka#producer-3| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT, 30 identical error(s) suppressed)
%3|1648348114.872|FAIL|rdkafka#producer-3| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT, 30 identical error(s) suppressed)
%3|1648348144.881|FAIL|rdkafka#producer-3| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT, 30 identical error(s) suppressed)
%3|1648348174.892|FAIL|rdkafka#producer-3| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect

In [ ]:
# create df that reads from Kafka
df_kf = spark\
        .readStream\
        .format("kafka")\
        .option("kafka.bootstrap.servers", "localhost:9092")\
        .option('subscribe', "Topic2")\
        .load()\
        .selectExpr("CAST(value AS STRING)", "CAST(timestamp AS STRING)")

# format sentences
df_kf_words = df_kf.select(explode(split(df_kf.value, " ")).alias('word'), 'timestamp')
df_kf_words_count = df_kf_words.groupBy("word", "timestamp").count().select('word', 'count','timestamp')

# create json with words
df_kf_words_final = df_kf_words_count.withColumn("words", to_json(struct("word","count", "timestamp"))).select("words")

# stream topics
query = df_kf_words_final \
    .writeStream \
    .outputMode('complete') \
    .format('console') \
    .trigger(processingTime='5 seconds') \
    .start()

query.awaitTermination()


22/03/27 02:37:27 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-8b3dce03-eb7a-4b90-87d4-c18d0a781e45. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/03/27 02:37:27 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
22/03/27 02:37:27 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-a712236e-eed7-426b-b5ac-1d6743585527-1391255647-driver-0-1, groupId=spark-kafka-source-a712236e-eed7-426b-b5ac-1d6743585527-1391255647-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
22/03/27 02:37:27 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-a712236e-eed7-426b-b5ac-1d6743585527-1391255647-driv

In [ ]:
from cassandra.cluster import Cluster
from kafka import KafkaProducer, KafkaConsumer

topic = 'Topic3'
producer = KafkaProducer(bootstrap_servers='localhost:9092')

producer.send(topic, b'test3 message')

In [ ]:
cluster = Cluster()
session = cluster.connect()

In [ ]:
def main2(broker_str, topic):
    try:
        consumer = KafkaConsumer(group_id='my-group', bootstrap_servers=['localhost:9092'], auto_offset_reset="latest")
        consumer.subscribe([topic])
        while True:
            try:
                k_msg = consumer.poll(timeout_ms=200)
            except OffsetOutOfRangeError:
                log.info("Offset out of range")
            else:
                if k_msg:
                    for msgs in consumer:
                        entry = json.loads(json.loads(message.value))['log']
                            print("word entry", entry)                        
    except:
        print("executution error")
